RE -> ε-NFA

(a+b)*abb
( recognize ( push , a recognize check len(stack) check stack[-1]+ a push ,+ push(just push) , b recognize check len(stack) check stack[-1]+ stack pop+ b push stack push+ , ) traverse until ( reveals check stack[index'('-1]+ check index'('0 pop all(just concat) , * check len(stack) write , a check len(stack)write, b write, b write

(b+a(aa*b)*b)* 
( recognize ( push, b recognize check len(stack) check stack[-1]+ b push , + push , a recognize check len(stack) check stack[-1]+ +pop 


(b+aa+ac+aaa+aac)*

(1+01)*00(0+1)*

(0+1)*011  


In [1524]:
def isapt(c):
    assert len(c) == 1
    if ord(c) >= ord('a') and ord(c) <= ord('z') or ord(c) >= ord('A') and ord(c) <= ord('Z') or ord(c) >= ord('0') and ord(c) <= ord('9'):
        return True
    return False

In [1525]:
def isfast(org, top):
    if org == '.' and top == '+':
        return True
    elif org == '*' and top != '*':
        return True
    return False
         

In [1526]:
class node:
    def __init__(self, cn, cv, left=None, right=None):
        # cn : 0:+ 1:. 2:* 3:symbols
        self.cn = cn
        self.cv = cv
        self.left = None
        self.right = None

In [1527]:
RE_ARR = ["(a+b)*abb", "(b+a(aa*b)*b)*" ,"(b+aa+ac+aaa+aac)*", "(1+01)*00(0+1)*" ,"(0+1)*011"]

In [1528]:
RE = RE_ARR[0]
RE = "(a+b)*abb" #여기 RE입력!
concatall = ""
str_arr = []

for i in range(len(RE)-1):
    str_arr.append(RE[i])
    if isapt(RE[i]):
        if isapt(RE[i+1]) or RE[i+1] == '(':
            str_arr.append('.')
    if (RE[i] == ')' or RE[i] == '*') and (RE[i+1] == '(' or isapt(RE[i+1])):
        str_arr.append('.')
str_arr.append(RE[-1])
concatall = "".join(str_arr)

print(concatall)

(a+b)*.a.b.b


In [1529]:
stack = []
post_arr = []

for c in concatall:
    if isapt(c) or c == "*":
        post_arr.append(c)
    elif c == ")":
        while len(stack) > 0 and stack[-1] != "(":
            post_arr.append(stack.pop())
        stack.pop()
    elif c == "(":
        stack.append(c)
    elif len(stack) == 0 or stack[-1] == "(" or isfast(c, stack[-1]):
        
        stack.append(c)
    else:
        while len(stack) > 0 and stack[-1] != "(" and not isfast(c, stack[-1]):
            post_arr.append(stack.pop())
        stack.append(c)
while len(stack) > 0:
    post_arr.append(stack.pop())

postall = "".join(post_arr)
print(postall)

ab+*a.b.b.


In [1530]:
stack = []
for c in postall:
        if c == "+":
            nd = node(0,c)
            nd.right = stack.pop()
            nd.left = stack.pop()
            stack.append(nd)
        elif c == ".":
            nd = node(1,c)
            nd.right = stack.pop()
            nd.left = stack.pop()
            stack.append(nd)
        elif c == "*":
            nd = node(2,c)
            nd.left = stack.pop() 
            stack.append(nd)
        elif c == "(" or c == ")":
            continue  
        else:
            stack.append(node(3,c))

In [1531]:
class eNFA:
    def __init__(self):
        self.next = {}

In [1532]:
def scan_plus(nd):
    start = eNFA()
    end = eNFA()

    left_eNFA = node_filter(nd.left)
    right_eNFA = node_filter(nd.right)

    start.next['ε'] = [left_eNFA[0], right_eNFA[0]]
    left_eNFA[1].next['ε'] = [end]
    right_eNFA[1].next['ε'] = [end]

    return start, end

def scan_dot(nd):
    left_nfa  = node_filter(nd.left)
    right_nfa = node_filter(nd.right)

    left_nfa[1].next['ε'] = [right_nfa[0]]
    return left_nfa[0], right_nfa[1]

def scan_star(nd):
    start = eNFA()
    end = eNFA()

    starred_nfa = node_filter(nd.left)

    start.next['ε'] = [starred_nfa[0], end]
    starred_nfa[1].next['ε'] = [starred_nfa[0], end]

    return start, end

def scan_symbol(nd):
    start = eNFA()
    end = eNFA()
    
    start.next[nd.cv] = [end]
    return start, end

In [1533]:
def node_filter(nd):
    if nd.cn == 0:
        return scan_plus(nd)
    elif nd.cn == 1:
        return scan_dot(nd)
    elif nd.cn == 2:
        return scan_star(nd)
    elif nd.cn == 3:
        return scan_symbol(nd)
    else:
        print("error")

In [1534]:
efas = node_filter(stack[0])

In [1535]:
print(efas)
enfa = {}

(<__main__.eNFA object at 0x7f9e58310bb0>, <__main__.eNFA object at 0x7f9e58310670>)


In [1536]:
def sonbug_numbering(integer):
    assert integer < 1000, "too complicate RE"
    if integer > 99:
        return str(integer)
    elif integer > 9:
        return "0" + str(integer)
    else:
        return "00"+str(integer)

In [1537]:
def make_enfa(State, States_history, symbols):
    if State in States_history:
        return
    States_history.append(State)
    for sb in list(State.next):
        if sb not in enfa['TerminalSet']:
            enfa['TerminalSet'].add(sb)
        for nest in State.next[sb]:
            if nest not in symbols:
                symbols[nest] = sorted(symbols.values())[-1]+1
                qs = "q" + sonbug_numbering(symbols[nest])
                enfa['StateSet'].add(qs)
            if not ("(q"+sonbug_numbering(symbols[State])+", "+sb+")" in enfa['DeltaFunctions']):
                enfa['DeltaFunctions']["(q"+sonbug_numbering(symbols[State])+", "+sb+")"] = set()
            enfa['DeltaFunctions']["(q"+sonbug_numbering(symbols[State])+", "+sb+")"].add("q"+sonbug_numbering(symbols[nest]))      
        for nest in State.next[sb]:
            make_enfa(nest, States_history, symbols)

In [1538]:
enfa['StateSet'] = set()
enfa['TerminalSet'] = set()
enfa['DeltaFunctions'] = {}
enfa['StartState'] = set()
enfa['FinalStateSet'] = set()
q_0 = "q000"
enfa['StateSet'].add(q_0)
make_enfa(efas[0], [], {efas[0]:0})
    

enfa["StartState"].add("q000")
for st in list(enfa["StateSet"]):
    count = 0
    for key,val in enfa['DeltaFunctions'].items():
        for value in list(val):
            if key[1:5] == st and value != st:
                count += 1
    if count == 0 and st not in enfa["FinalStateSet"]:
        enfa["FinalStateSet"].add(st)


In [1539]:
print(enfa)

{'StateSet': {'q004', 'q010', 'q011', 'q009', 'q000', 'q001', 'q006', 'q012', 'q013', 'q003', 'q005', 'q002', 'q008', 'q007'}, 'TerminalSet': {'b', 'ε', 'a'}, 'DeltaFunctions': {'(q000, ε)': {'q002', 'q001'}, '(q001, ε)': {'q004', 'q003'}, '(q003, a)': {'q005'}, '(q005, ε)': {'q006'}, '(q006, ε)': {'q002', 'q001'}, '(q002, ε)': {'q007'}, '(q007, a)': {'q008'}, '(q008, ε)': {'q009'}, '(q009, b)': {'q010'}, '(q010, ε)': {'q011'}, '(q011, b)': {'q012'}, '(q004, b)': {'q013'}, '(q013, ε)': {'q006'}}, 'StartState': {'q000'}, 'FinalStateSet': {'q012'}}


In [1540]:
from pprint import pprint
pprint(enfa)

{'DeltaFunctions': {'(q000, ε)': {'q002', 'q001'},
                    '(q001, ε)': {'q004', 'q003'},
                    '(q002, ε)': {'q007'},
                    '(q003, a)': {'q005'},
                    '(q004, b)': {'q013'},
                    '(q005, ε)': {'q006'},
                    '(q006, ε)': {'q002', 'q001'},
                    '(q007, a)': {'q008'},
                    '(q008, ε)': {'q009'},
                    '(q009, b)': {'q010'},
                    '(q010, ε)': {'q011'},
                    '(q011, b)': {'q012'},
                    '(q013, ε)': {'q006'}},
 'FinalStateSet': {'q012'},
 'StartState': {'q000'},
 'StateSet': {'q000',
              'q001',
              'q002',
              'q003',
              'q004',
              'q005',
              'q006',
              'q007',
              'q008',
              'q009',
              'q010',
              'q011',
              'q012',
              'q013'},
 'TerminalSet': {'b', 'ε', 'a'}}


ε-NFA -> DFA

In [1541]:
from copy import deepcopy

In [1542]:
dfa = {}

In [1543]:
def listunion(list1, list2):
    return sorted(list(set(list1) | set(list2)))

def get_enfa_delta(state, input):
    if "("+state+", "+input+")" in enfa['DeltaFunctions'].keys():
        return enfa['DeltaFunctions']["("+state+", "+input+")"]
    return False 

In [1544]:
dfa['StateSet'] = set()
dfa['TerminalSet'] = set()
dfa['DeltaFunctions'] = {}
dfa['StartState'] = set()
dfa['FinalStateSet'] = set()




candidate = []


for state in enfa['StateSet']: 
    for sigma in enfa['TerminalSet']:
        if sigma != 'ε':
            continue
        q_temp = []
        
        for key,val in enfa['DeltaFunctions'].items():
            for value in val:
                start = key[1:5]
                inp = key[7]
                end = value
                if state == start and sigma == inp:
                    if end not in q_temp:
                        q_temp.append(end)
        if not len(q_temp):
            continue
        q_states = []
        
        q_states.append(state)
        q_states = sorted(q_states)
        q_temp = sorted(q_temp)
        candidate.append([q_states, sigma, q_temp])    


e_match = {}


for i in enfa['StateSet']:
    e_match[tuple([i])] = [i]


no_change = False
while not no_change:
    no_change = True
    for delta in candidate: #single
        if len(delta[0]) == 1 and delta[1] == 'ε' and len(e_match[tuple(delta[0])]) == 1:
            e_match[tuple(delta[0])] = e_match[tuple(delta[0])]+delta[2] # ('q000',) : ["q001","q002"]
            if delta[0][0] not in e_match[tuple(delta[0])]: #본인 넣는다.
                e_match[tuple(delta[0])].append(delta[0][0])
            no_change = False
        if len(delta[0]) == 1 and delta[1] == 'ε':
            temp = deepcopy(e_match[tuple(delta[0])])
            
            for get in e_match[tuple(delta[0])]:
                
                if (get,) in e_match.keys(): 
                    temp = listunion(e_match[(get,)],temp)     
            if len(temp) > len(e_match[tuple(delta[0])]):
                e_match[tuple(delta[0])] = deepcopy(temp) 
                no_change = False

def multcandidate(states):

    for sigma in enfa['TerminalSet']:
        if sigma != 'ε':
            continue
        q_temp = []
        for state in states:
            for key,val in enfa['DeltaFunctions'].items():
                for value in val:
                    start = key[1:5]
                    inp = key[7]
                    end = value
                    if state == start and sigma == inp:
                        if end not in q_temp:
                            q_temp.append(end)
        if not len(q_temp):
            continue

        
        q_states = states
        q_states = sorted(q_states)
        q_temp = sorted(q_temp)
        candidate.append([q_states, sigma, q_temp])  

    no_change = False
    while not no_change:
        no_change = True
        for delta in candidate: #mult
            
            if len(delta[0]) != 1 and delta[1] == 'ε' and tuple(delta[0]) not in e_match.keys():
                e_match[tuple(delta[0])] = deepcopy(delta[2]) # ('q000','q001') : ["q001","q002"]
                no_change = False
            if len(delta[0]) != 1 and delta[1] == 'ε':
                temp = deepcopy(e_match[tuple(delta[0])])
                for d in delta[0]:
                    for get in e_match[(d,)]:
                        if (get,) in e_match.keys(): 
                            temp=listunion(e_match[(get,)],temp)
                if len(temp) > len(e_match[tuple(delta[0])]):
                    e_match[tuple(delta[0])] = deepcopy(temp) 
                    no_change = False



new_candidate = []
for ss in enfa['StartState']:
    dfa['StartState'].add(tuple(sorted(e_match[(ss,)])))
    dfa['StateSet'].add(tuple(sorted(e_match[(ss,)])))
    no_change = False
    have_to_do = []
    have_to_do.append((ss,))
    while len(have_to_do):
        tg = have_to_do.pop()
        
        for sigma in enfa['TerminalSet']:
            if sigma == 'ε':
                continue
            after_closer_result = []
            
            if tuple(tg) not in e_match.keys():
                
                multcandidate(tg)

            for ecs in e_match[tuple(tg)]:
                
                result = get_enfa_delta(ecs, sigma)
                if result:

                    for r in result:
                        if (r,) in e_match.keys():
                            after_closer_result = listunion(after_closer_result, e_match[(r,)])
            if not len(after_closer_result): # 결과가 없을떄
                continue 
            if tuple(after_closer_result) not in dfa['StateSet']:
                dfa['StateSet'].add(tuple(after_closer_result))
                have_to_do.append(after_closer_result)
            new_candidate.append([sorted(e_match[tuple(tg)]),sigma,sorted(after_closer_result)])



for states in dfa['StateSet']:
    for state in states:
        if state in enfa['FinalStateSet'] and states not in dfa['FinalStateSet']:
            dfa['FinalStateSet'].add(states)

enfa['TerminalSet'].remove('ε')
dfa['TerminalSet'] = deepcopy(enfa['TerminalSet'])



In [1545]:
pprint(new_candidate)

[[['q000', 'q001', 'q002', 'q003', 'q004', 'q007'],
  'b',
  ['q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q013']],
 [['q000', 'q001', 'q002', 'q003', 'q004', 'q007'],
  'a',
  ['q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009']],
 [['q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009'],
  'b',
  ['q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q010', 'q011', 'q013']],
 [['q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009'],
  'a',
  ['q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009']],
 [['q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q010', 'q011', 'q013'],
  'b',
  ['q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q012', 'q013']],
 [['q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q010', 'q011', 'q013'],
  'a',
  ['q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009']],
 [['q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q012', 'q013'],
  'b',
  ['q001', 'q002', 'q003', 'q004', 'q

In [1546]:
pprint(dfa)

{'DeltaFunctions': {},
 'FinalStateSet': {('q001',
                    'q002',
                    'q003',
                    'q004',
                    'q006',
                    'q007',
                    'q012',
                    'q013')},
 'StartState': {('q000', 'q001', 'q002', 'q003', 'q004', 'q007')},
 'StateSet': {('q000', 'q001', 'q002', 'q003', 'q004', 'q007'),
              ('q001',
               'q002',
               'q003',
               'q004',
               'q005',
               'q006',
               'q007',
               'q008',
               'q009'),
              ('q001',
               'q002',
               'q003',
               'q004',
               'q006',
               'q007',
               'q010',
               'q011',
               'q013'),
              ('q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q012', 'q013'),
              ('q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q013')},
 'TerminalSet': {'b', 'a'}}


In [1547]:
reduce_table = {}
for i,v in enumerate(dfa['StateSet']):
    val = "q"+sonbug_numbering(i)
    reduce_table[v] = val
print(reduce_table)

{('q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q013'): 'q000', ('q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q010', 'q011', 'q013'): 'q001', ('q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009'): 'q002', ('q000', 'q001', 'q002', 'q003', 'q004', 'q007'): 'q003', ('q001', 'q002', 'q003', 'q004', 'q006', 'q007', 'q012', 'q013'): 'q004'}


In [1548]:
for delta in new_candidate:
    dfa['DeltaFunctions']["("+reduce_table[tuple(sorted(delta[0]))]+", "+delta[1]+")"] = {reduce_table[tuple(sorted(delta[2]))]}

sett = set()
sett.add(reduce_table[list(dfa['StartState'])[0]])

dfa['StartState'] = deepcopy(sett)

sett = set()

for state in dfa['StateSet']:
    sett.add(reduce_table[state])
    
dfa['StateSet'] = deepcopy(sett)

sett = set()

for state in dfa['FinalStateSet']:
    sett.add(reduce_table[state])
dfa['FinalStateSet'] = sett


In [1549]:
pprint(dfa)

{'DeltaFunctions': {'(q000, a)': {'q002'},
                    '(q000, b)': {'q000'},
                    '(q001, a)': {'q002'},
                    '(q001, b)': {'q004'},
                    '(q002, a)': {'q002'},
                    '(q002, b)': {'q001'},
                    '(q003, a)': {'q002'},
                    '(q003, b)': {'q000'},
                    '(q004, a)': {'q002'},
                    '(q004, b)': {'q000'}},
 'FinalStateSet': {'q004'},
 'StartState': {'q003'},
 'StateSet': {'q004', 'q000', 'q001', 'q003', 'q002'},
 'TerminalSet': {'b', 'a'}}


DFA -> reduced DFA

In [1550]:
def get_dfa_delta(state, input):
    if "("+state+", "+input+")" in dfa['DeltaFunctions'].keys():
        return list(dfa['DeltaFunctions']["("+state+", "+input+")"])[0]
    return "-1"

def something_in_list(something, list):
    if something in list:
        return True
    return False 

def combine_same_inlist(lst):
    pass

from collections import defaultdict

In [1551]:


getarr=sorted(list(dfa['StateSet']))

arr = [list(dfa['StateSet'] - dfa['FinalStateSet']),sorted(list(dfa['FinalStateSet']))]
LEN_STATE = len(dfa['StateSet'])
checkarr = [["I"]*len(dfa['TerminalSet']) for _ in range(LEN_STATE)]

for i in range(LEN_STATE+1):
    lenarr = len(arr)
    for j in range(LEN_STATE):
        for ki,kv in enumerate(dfa['TerminalSet']):
            anstr=get_dfa_delta(getarr[j],kv)
            
            
            if anstr=="-1":
                checkarr[j][ki] = "N"
            else:
                for mi,mv in enumerate(arr):
                    if something_in_list(anstr,mv):
                        checkarr[j][ki] = str(mi)
    temp_arr = []

    
    big_ddic = []
    for ji,jv in enumerate(arr):
        ddic = defaultdict(list)
        for ki,kv in enumerate(jv):
            sumstr = ""
            for m in checkarr[int(kv[1:])]:
                sumstr += m
            ddic[sumstr].append(kv)
        
        for key,val in ddic.items():
            temp_arr.append(val)
        big_ddic.append(deepcopy(ddic))
        


    arr = deepcopy(temp_arr)    
    
    
    if lenarr == len(arr):
        print(arr)
        print(big_ddic)
        break




[['q003', 'q000'], ['q002'], ['q001'], ['q004']]
[defaultdict(<class 'list'>, {'01': ['q003', 'q000']}), defaultdict(<class 'list'>, {'21': ['q002']}), defaultdict(<class 'list'>, {'31': ['q001']}), defaultdict(<class 'list'>, {'01': ['q004']})]


In [1552]:
new_candidate = []
for i in big_ddic:
    for k,v in i.items():
        for ji,jv in enumerate(dfa['TerminalSet']):
            #v가 jv를 보고 arr[k[ji]]로 이동한다는 것임
            new_candidate.append([v,jv,arr[int(k[ji])]])
print(new_candidate)


[[['q003', 'q000'], 'b', ['q003', 'q000']], [['q003', 'q000'], 'a', ['q002']], [['q002'], 'b', ['q001']], [['q002'], 'a', ['q002']], [['q001'], 'b', ['q004']], [['q001'], 'a', ['q002']], [['q004'], 'b', ['q003', 'q000']], [['q004'], 'a', ['q002']]]


In [1553]:
rdfa = {}
rdfa['StateSet'] = set()
rdfa['TerminalSet'] = deepcopy(dfa['TerminalSet'])
rdfa['DeltaFunctions'] = {}
rdfa['StartState'] = set()
rdfa['FinalStateSet'] = set()

reduce_table = {}
for i,v in enumerate(arr):
    val = "q"+sonbug_numbering(i)
    reduce_table[tuple(v)] = val
print(reduce_table)

{('q003', 'q000'): 'q000', ('q002',): 'q001', ('q001',): 'q002', ('q004',): 'q003'}


In [1554]:
for delta in new_candidate:
    rdfa['DeltaFunctions']["("+reduce_table[tuple(delta[0])]+", "+delta[1]+")"] = {reduce_table[tuple(delta[2])]}

sett = set()

for states in arr:
    sett.add(reduce_table[tuple(states)])

rdfa['StateSet'] = deepcopy(sett)

sett = set()

for ss in dfa['StartState']:
    for k,v in reduce_table.items():
        if ss in k:
            sett.add(v)

rdfa['StartState'] = deepcopy(sett)

sett = set()

for fs in dfa['FinalStateSet']:
    for k,v in reduce_table.items():
        if fs in k:
            sett.add(v)

rdfa['FinalStateSet'] = deepcopy(sett)



In [1555]:
pprint(rdfa)

{'DeltaFunctions': {'(q000, a)': {'q001'},
                    '(q000, b)': {'q000'},
                    '(q001, a)': {'q001'},
                    '(q001, b)': {'q002'},
                    '(q002, a)': {'q001'},
                    '(q002, b)': {'q003'},
                    '(q003, a)': {'q001'},
                    '(q003, b)': {'q000'}},
 'FinalStateSet': {'q003'},
 'StartState': {'q000'},
 'StateSet': {'q001', 'q002', 'q000', 'q003'},
 'TerminalSet': {'b', 'a'}}
